# Part 2 - Prompt Engineering

### Overview

This notebook demonstrates using the general-purpose GPT model for text classification.

The following steps are covered:
* Creating system prompt
* Using the Open AI ChatCompletion API
* Testing text classification

### Resources

* https://platform.openai.com/docs/guides/prompt-engineering
* https://platform.openai.com/docs/api-reference/chat


In [47]:
# %pip install openai --upgrade
# %pip show openai
# %pip install tiktoken

In [48]:
import pandas as pd
from openai import OpenAI
import os
import json
from src.util import distributionPreservingDownsample

In [49]:
from dotenv import load_dotenv; load_dotenv()
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Check Token Requirements

Based on our expected completions, compute how many tokens are needed to encode those completions.  Then we set the `max_tokens` accordingly when using the API.

In [50]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-1106")

spamCompletion = 'Spam'
hamCompletion = 'Ham'

spamTokens = encoding.encode(spamCompletion)
hamTokens = encoding.encode(hamCompletion)

print(f'Spam token count: {len(spamTokens)}')
print(f'Ham token count: {len(hamTokens)}')

Spam token count: 2
Ham token count: 1


# Set Up API

In [51]:

def isSpam(message, systemPrompts):
    messages = [{"role": "system", "content": systemPrompt} for systemPrompt in systemPrompts]
    messages.append({"role": "user", "content": message})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        temperature=0.5,
        max_tokens=2
    )
    # print("Completion: {}".format(response.choices[0]))
    return response.choices[0].message.content == spamCompletion

# Test a Couple Examples

In [52]:
defaultSystemPrompt = """
You will be provided with a text message. 
You will need to classify the text message as spam, ham. 
Spam is a text message that is spam, harmful, abusive, or otherwise unwanted. 
Ham is a text message that is not spam. 
Response should be a single word: Spam or Ham.
"""

In [53]:
isSpam("Hey what's happening? Want to get some ramen?", [defaultSystemPrompt])

False

In [54]:
isSpam("Do you need help recovering your lost crypto tokens? We recover quickly with full privacy. Please send your private key and we will get started.", [defaultSystemPrompt])

True

# Try some alternate prompts

In [55]:
defaultSystemPrompt = """
You are a spam classifier. 
Your job is to examine the content of a message and determine if it is spam or ham. 
Spam is a message that is spam, harmful, abusive, an attempt to access private information, harrassment, or otherwise unwanted. 
Ham is any message that is not classified as spam.
Response should be a single word: Spam or Ham.
"""
exampleSpam1 = 'Example Spam: "Hey bro, are you looking for a quick way to double your money with crypto?"'
exampleSpam2 = 'Example Spam: "This is the IRS. There is a warrant out for your arrest due to an unpaid balance of $75,000 USD. We have already contacted the Federal Marshalls with regards to this matter. Please call us back at 1-800-555-5555 with payment information available or you will go to jail."'


In [56]:
isSpam("Hey what's happening? Want to get some ramen?", [defaultSystemPrompt, exampleSpam1, exampleSpam2])

False

In [57]:
isSpam("Do you need help recovering your lost crypto tokens? We recover quickly with full privacy. Please send your private key and we will get started.", [defaultSystemPrompt, exampleSpam1, exampleSpam2])

True

# Exercise

Can you craft a message that is incorrectly classified by the detector?

If so, can you update the prompt so it correctly classifies the message?